**Step 1. Import the Libraries**

In [ ]:
!pip install pandas numpy

import pandas as pd
import numpy as np

print("Libraries are succesfully imported!")


Libraries are succesfully imported!


**Step 2. Import the CSV File**

In [ ]:
df = pd.read_csv('survey_data_updated.csv')

**Step 3. Control the Data**

In [ ]:
print('First few records::', df.head(10))
print('Column names:', df.columns)
print('Row count:',df.shape[0])
print('Column count:', df.shape[1])

First few records::    ResponseId                                         MainBranch  \
0           2                     I am a developer by profession   
1           3                     I am a developer by profession   
2          10                     I am a developer by profession   
3          11  I used to be a developer by profession, but no...   
4          12                     I am a developer by profession   
5          19                     I am a developer by profession   
6          23                     I am a developer by profession   
7          24                     I am a developer by profession   
8          27                     I am a developer by profession   
9          29                     I am a developer by profession   

               Age                                         Employment  \
0  35-44 years old                                Employed, full-time   
1  45-54 years old                                Employed, full-time   
2  35-44 yea

**Step 4. Check The Columns That Are Important For The Dasboard**

In [ ]:
columns_to_check = [
    'LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith', 'PlatformHaveWorkedWith',
    'WebframeHaveWorkedWith', 'LanguageWantToWorkWith', 'DatabaseWantToWorkWith',
    'PlatformWantToWorkWith', 'WebframeWantToWorkWith', 'Age', 'Country', 'EdLevel'
]

print("Checking required columns:")
for col in columns_to_check:
    if col in df.columns:
        print(f"✓ {col}")
    else:
        print(f"✗ {col} - NOT FOUND")

print("\nSample data from key columns:")
for col in columns_to_check:
    if col in df.columns:
        print(f"\n{col}: {df[col].dropna().iloc[0] if not df[col].dropna().empty else 'No data'}")

Checking required columns:
✓ LanguageHaveWorkedWith
✓ DatabaseHaveWorkedWith
✓ PlatformHaveWorkedWith
✓ WebframeHaveWorkedWith
✓ LanguageWantToWorkWith
✓ DatabaseWantToWorkWith
✓ PlatformWantToWorkWith
✓ WebframeWantToWorkWith
✓ Age
✓ Country
✓ EdLevel

Sample data from key columns:

LanguageHaveWorkedWith: Bash/Shell (all shells);Go;HTML/CSS;Java;JavaScript;Python;TypeScript

DatabaseHaveWorkedWith: Dynamodb;MongoDB;PostgreSQL

PlatformHaveWorkedWith: Amazon Web Services (AWS);Heroku;Netlify

WebframeHaveWorkedWith: Express;Next.js;Node.js;React

LanguageWantToWorkWith: Bash/Shell (all shells);Go;HTML/CSS;Java;JavaScript;Kotlin;Python;TypeScript

DatabaseWantToWorkWith: PostgreSQL

PlatformWantToWorkWith: Amazon Web Services (AWS);Heroku;Netlify

WebframeWantToWorkWith: Express;Htmx;Node.js;React;Remix

Age: 35-44 years old

Country: United Kingdom of Great Britain and Northern Ireland

EdLevel: Bachelor’s degree (B.A., B.S., B.Eng., etc.)


As can be seen, for the "...WorkedWith" columns there are more than one answer in one record seperated with semicolon. This creates an error in the dashboarding process. We need to work on these columns and split the answers.

**Step 5. Keep the Desired Columns and Saved for the Dashboard 3 (Demographic Dashboard)**


In [ ]:
desired_columns = [
    'ResponseId', 'LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith', 'PlatformHaveWorkedWith',
    'WebframeHaveWorkedWith', 'LanguageWantToWorkWith', 'DatabaseWantToWorkWith',
    'PlatformWantToWorkWith', 'WebframeWantToWorkWith', 'Age', 'Country', 'EdLevel'
]

df_dashboard3 = df[desired_columns].copy()

print("Data Dimension:", df_dashboard3.shape)
print("Remaining Columns:", df_dashboard3.columns.tolist())

df_dashboard3.to_csv('dashboard3_data.csv', index=False)
print("dashboard3_data.csv saved")

Data Dimension: (18845, 12)
Remaining Columns: ['ResponseId', 'LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith', 'PlatformHaveWorkedWith', 'WebframeHaveWorkedWith', 'LanguageWantToWorkWith', 'DatabaseWantToWorkWith', 'PlatformWantToWorkWith', 'WebframeWantToWorkWith', 'Age', 'Country', 'EdLevel']
dashboard3_data.csv saved


**Step 6. Remove the Demographic Columns and Apply Split for the Dashboard 1 & 2 (Current and Future Technology)**

In [ ]:
columns_to_drop = ['Age', 'Country', 'EdLevel']
df_cleaned = df_dashboard3.drop(columns=columns_to_drop, errors='ignore')

print("Demograhic columns are removed. New data dimensions:", df_cleaned.shape)
print("Remaning Columns:", df_cleaned.columns.tolist())

Demograhic columns are removed. New data dimensions: (18845, 9)
Remaning Columns: ['ResponseId', 'LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith', 'PlatformHaveWorkedWith', 'WebframeHaveWorkedWith', 'LanguageWantToWorkWith', 'DatabaseWantToWorkWith', 'PlatformWantToWorkWith', 'WebframeWantToWorkWith']


In [ ]:
def split_multi_value_columns(df, multi_value_columns):
    """
    Separates multi-value columns into separate rows, ResponseId is preserved.
    """
    result_dfs = []

    for column in multi_value_columns:
        if column in df.columns:
            print(f"📊 {column} processing...")

            # NaN cleaning
            temp_df = df[['ResponseId', column]].dropna()

            # Split with ; and create seperate record (row) for each value
            temp_df[column] = temp_df[column].str.split(';')
            exploded_df = temp_df.explode(column)

            # Clean the missing values
            exploded_df[column] = exploded_df[column].str.strip()
            exploded_df = exploded_df[exploded_df[column] != '']

            result_dfs.append(exploded_df)

    return result_dfs

# Multi-value columns to be processed
multi_value_columns = [
    'LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith', 'PlatformHaveWorkedWith',
    'WebframeHaveWorkedWith', 'LanguageWantToWorkWith', 'DatabaseWantToWorkWith',
    'PlatformWantToWorkWith', 'WebframeWantToWorkWith'
]

# Create seperate data frames for each column
split_dataframes = split_multi_value_columns(df_cleaned, multi_value_columns)

# Save each data frame as different CSV files
for i, split_df in enumerate(split_dataframes):
    column_name = multi_value_columns[i]
    filename = f"split_{column_name}.csv"
    split_df.to_csv(filename, index=False)
    print(f"✅ {filename} is saved: {len(split_df)} records")

📊 LanguageHaveWorkedWith processing...
📊 DatabaseHaveWorkedWith processing...
📊 PlatformHaveWorkedWith processing...
📊 WebframeHaveWorkedWith processing...
📊 LanguageWantToWorkWith processing...
📊 DatabaseWantToWorkWith processing...
📊 PlatformWantToWorkWith processing...
📊 WebframeWantToWorkWith processing...
✅ split_LanguageHaveWorkedWith.csv is saved: 116557 records
✅ split_DatabaseHaveWorkedWith.csv is saved: 69586 records
✅ split_PlatformHaveWorkedWith.csv is saved: 50655 records
✅ split_WebframeHaveWorkedWith.csv is saved: 77803 records
✅ split_LanguageWantToWorkWith.csv is saved: 106356 records
✅ split_DatabaseWantToWorkWith.csv is saved: 65913 records
✅ split_PlatformWantToWorkWith.csv is saved: 48410 records
✅ split_WebframeWantToWorkWith.csv is saved: 71417 records


**Step 7. Control the CSV Files**

In [30]:
import pandas as pd
import glob

# find the CSV files
csv_files = glob.glob("*.csv")
print("CSV Files:")
for file in csv_files:
    print(f"• {file}")

print("\n" + "="*80 + "\n")

# read the CSV files and control with the first few rows
for file in csv_files:
    print(f"{file} - First 10 Record:")
    print("-" * 60)

    try:
        df = pd.read_csv(file)

        print(f"Dimension: {df.shape[0]} rows, {df.shape[1]} columns")
        print(f"Columns: {list(df.columns)}")

        if len(df) > 0:
            display(df.head(10))
        else:
            print("This file is empty!")

    except Exception as e:
        print(f"Error: {str(e)}")

    print("\n" + "="*80 + "\n")

CSV Files:
• dashboard3_data.csv
• survey_data_updated.csv
• split_DatabaseWantToWorkWith.csv
• split_PlatformWantToWorkWith.csv
• split_PlatformHaveWorkedWith.csv
• split_LanguageHaveWorkedWith.csv
• split_LanguageWantToWorkWith.csv
• split_DatabaseHaveWorkedWith.csv
• split_WebframeWantToWorkWith.csv
• split_WebframeHaveWorkedWith.csv


dashboard3_data.csv - First 10 Record:
------------------------------------------------------------
Dimension: 18845 rows, 12 columns
Columns: ['ResponseId', 'LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith', 'PlatformHaveWorkedWith', 'WebframeHaveWorkedWith', 'LanguageWantToWorkWith', 'DatabaseWantToWorkWith', 'PlatformWantToWorkWith', 'WebframeWantToWorkWith', 'Age', 'Country', 'EdLevel']


,ResponseId,LanguageHaveWorkedWith,DatabaseHaveWorkedWith,PlatformHaveWorkedWith,WebframeHaveWorkedWith,LanguageWantToWorkWith,DatabaseWantToWorkWith,PlatformWantToWorkWith,WebframeWantToWorkWith,Age,Country,EdLevel
0,2,Bash/Shell (all shells);Go;HTML/CSS;Java;JavaS...,Dynamodb;MongoDB;PostgreSQL,Amazon Web Services (AWS);Heroku;Netlify,Express;Next.js;Node.js;React,Bash/Shell (all shells);Go;HTML/CSS;Java;JavaS...,PostgreSQL,Amazon Web Services (AWS);Heroku;Netlify,Express;Htmx;Node.js;React;Remix,35-44 years old,United Kingdom of Great Britain and Northern I...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)"
1,3,C#,Firebase Realtime Database,Google Cloud,ASP.NET CORE,C#,Firebase Realtime Database,Google Cloud,ASP.NET CORE,45-54 years old,United Kingdom of Great Britain and Northern I...,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)"
2,10,HTML/CSS;JavaScript;TypeScript,MongoDB;MySQL;PostgreSQL;Redis,Amazon Web Services (AWS);Digital Ocean;Fireba...,Express;Next.js;Node.js;React,HTML/CSS;JavaScript;TypeScript,MongoDB;MySQL;PostgreSQL;Redis,Amazon Web Services (AWS);Digital Ocean;Fireba...,Express;Next.js;Node.js;React,35-44 years old,Serbia,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)"
3,11,JavaScript;Python;Ruby,Elasticsearch;MongoDB;PostgreSQL;SQLite,Amazon Web Services (AWS);Google Cloud,Node.js;React;Ruby on Rails,C#;Crystal;Java;JavaScript;Python;Ruby,Elasticsearch;MongoDB;PostgreSQL;SQLite,Google Cloud,Django;Node.js;React;Ruby on Rails,35-44 years old,United States of America,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)"
4,12,C;C#;C++;Delphi;HTML/CSS;Java;JavaScript;Power...,Microsoft Access;Microsoft SQL Server;PostgreS...,Cloudflare,ASP.NET;ASP.NET CORE;Express;jQuery;Node.js;React,C#;HTML/CSS;JavaScript;PowerShell;Rust;SQL;Typ...,Microsoft SQL Server;PostgreSQL;SQLite,Cloudflare,ASP.NET;ASP.NET CORE;Express;jQuery;Node.js;React,45-54 years old,Poland,"Professional degree (JD, MD, Ph.D, Ed.D, etc.)"
5,19,C#;HTML/CSS;JavaScript;PHP;Python;SQL,Microsoft Access;Microsoft SQL Server;MySQL;SQ...,Microsoft Azure,ASP.NET;ASP.NET CORE;Blazor;jQuery,C#;HTML/CSS;JavaScript;SQL,Microsoft SQL Server;SQLite,Microsoft Azure,ASP.NET CORE;Blazor,25-34 years old,Germany,Some college/university study without earning ...
6,23,Bash/Shell (all shells);C#;F#;JavaScript;Power...,Redis;SQLite,Amazon Web Services (AWS),ASP.NET CORE,Bash/Shell (all shells);C#;F#;Go,Redis;SQLite,Amazon Web Services (AWS);Cloudflare;Microsoft...,ASP.NET CORE,45-54 years old,Ireland,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)"
7,24,HTML/CSS;JavaScript;PHP;Python;SQL,MySQL;PostgreSQL;SQLite,Amazon Web Services (AWS),Django;jQuery;WordPress,HTML/CSS;JavaScript;PHP;Python;Rust;SQL,MySQL;PostgreSQL;SQLite,Amazon Web Services (AWS),Django;jQuery;WordPress,55-64 years old,United States of America,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)"
8,27,C#;HTML/CSS;JavaScript;Python;Scala;SQL,Microsoft SQL Server;PostgreSQL;Presto,Amazon Web Services (AWS),ASP.NET;Node.js,C#;JavaScript;Python;Scala;SQL,PostgreSQL;Presto,Amazon Web Services (AWS),Node.js,35-44 years old,Italy,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)"
9,29,Bash/Shell (all shells);C#;HTML/CSS;JavaScript...,MariaDB;Microsoft SQL Server;MySQL,Amazon Web Services (AWS);Hetzner;Microsoft Az...,ASP.NET;ASP.NET CORE;jQuery;Node.js;WordPress,Bash/Shell (all shells);C#;HTML/CSS;JavaScript...,MariaDB;Microsoft SQL Server;MySQL,Amazon Web Services (AWS);Colocation;Hetzner;M...,Angular;AngularJS;ASP.NET;ASP.NET CORE;jQuery;...,25-34 years old,United Kingdom of Great Britain and Northern I...,"Secondary school (e.g. American high school, G..."




survey_data_updated.csv - First 10 Record:
------------------------------------------------------------
Dimension: 18845 rows, 114 columns
Columns: ['ResponseId', 'MainBranch', 'Age', 'Employment', 'RemoteWork', 'Check', 'CodingActivities', 'EdLevel', 'LearnCode', 'LearnCodeOnline', 'TechDoc', 'YearsCode', 'YearsCodePro', 'DevType', 'OrgSize', 'PurchaseInfluence', 'BuyNewTool', 'BuildvsBuy', 'TechEndorse', 'Country', 'Currency', 'CompTotal', 'LanguageHaveWorkedWith', 'LanguageWantToWorkWith', 'LanguageAdmired', 'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith', 'DatabaseAdmired', 'PlatformHaveWorkedWith', 'PlatformWantToWorkWith', 'PlatformAdmired', 'WebframeHaveWorkedWith', 'WebframeWantToWorkWith', 'WebframeAdmired', 'EmbeddedHaveWorkedWith', 'EmbeddedWantToWorkWith', 'EmbeddedAdmired', 'MiscTechHaveWorkedWith', 'MiscTechWantToWorkWith', 'MiscTechAdmired', 'ToolsTechHaveWorkedWith', 'ToolsTechWantToWorkWith', 'ToolsTechAdmired', 'NEWCollabToolsHaveWorkedWith', 'NEWCollabToolsWantT

,ResponseId,MainBranch,Age,Employment,RemoteWork,Check,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,...,JobSatPoints_6,JobSatPoints_7,JobSatPoints_8,JobSatPoints_9,JobSatPoints_10,JobSatPoints_11,SurveyLength,SurveyEase,ConvertedCompYearly,JobSat
0,2,I am a developer by profession,35-44 years old,"Employed, full-time",Remote,Apples,Hobby;Contribute to open-source projects;Other...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;On the job tr...,Technical documentation;Blogs;Books;Written Tu...,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,3,I am a developer by profession,45-54 years old,"Employed, full-time",Remote,Apples,Hobby;Contribute to open-source projects;Other...,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Books / Physical media;Colleague;On the job tr...,Technical documentation;Blogs;Books;Written Tu...,...,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Easy,NaN,NaN
2,10,I am a developer by profession,35-44 years old,"Independent contractor, freelancer, or self-em...",Remote,Apples,Bootstrapping a business,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",On the job training;Other online resources (e....,Technical documentation;Blogs;Written Tutorial...,...,NaN,NaN,NaN,NaN,NaN,NaN,Too long,Easy,NaN,NaN
3,11,"I used to be a developer by profession, but no...",35-44 years old,"Employed, full-time",Remote,Apples,Hobby;Contribute to open-source projects,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Other online resources ...,Technical documentation;Books;Written Tutorial...,...,25.0,10.0,0.0,15.0,0.0,0.0,Appropriate in length,Easy,NaN,8.0
4,12,I am a developer by profession,45-54 years old,"Employed, full-time",In-person,Apples,Hobby;School or academic work,"Professional degree (JD, MD, Ph.D, Ed.D, etc.)","Books / Physical media;School (i.e., Universit...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Neither easy nor difficult,NaN,NaN
5,19,I am a developer by profession,25-34 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Apples,Hobby;Professional development or self-paced l...,Some college/university study without earning ...,Books / Physical media;On the job training;Oth...,Technical documentation;Blogs;Books;Stack Over...,...,60.0,10.0,10.0,0.0,0.0,0.0,Appropriate in length,Easy,NaN,10.0
6,23,I am a developer by profession,45-54 years old,"Employed, full-time",Remote,Apples,Hobby;Contribute to open-source projects;Profe...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",On the job training;Other online resources (e....,Technical documentation;Blogs;Stack Overflow;AI,...,20.0,20.0,0.0,20.0,0.0,0.0,Appropriate in length,Neither easy nor difficult,NaN,9.0
7,24,I am a developer by profession,55-64 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Apples,Contribute to open-source projects;Other (plea...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Other online resources (e.g., videos, blogs, f...",Technical documentation;Blogs;Stack Overflow;S...,...,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Neither easy nor difficult,NaN,NaN
8,27,I am a developer by profession,35-44 years old,"Employed, full-time",Remote,Apples,Professional development or self-paced learnin...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;On the job tr...,NaN,...,25.0,20.0,10.0,5.0,10.0,10.0,Appropriate in length,Neither easy nor difficult,NaN,7.0
9,29,I am a developer by profession,25-34 years old,"Independent contractor, freelancer, or self-em...",Remote,Apples,Hobby;Other (please specify):;Bootstrapping a ...,"Secondary school (e.g. American high school, G...",On the job training;Other online resources (e....,Technical documentation;Written Tutorials;Stac...,...,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Easy,NaN,NaN




split_DatabaseWantToWorkWith.csv - First 10 Record:
------------------------------------------------------------
Dimension: 65913 rows, 2 columns
Columns: ['ResponseId', 'DatabaseWantToWorkWith']


...

,ResponseId,DatabaseHaveWorkedWith
0,2,Dynamodb
1,2,MongoDB
2,2,PostgreSQL
3,3,Firebase Realtime Database
4,10,MongoDB
5,10,MySQL
6,10,PostgreSQL
7,10,Redis
8,11,Elasticsearch
9,11,MongoDB




split_WebframeWantToWorkWith.csv - First 10 Record:
------------------------------------------------------------
Dimension: 71417 rows, 2 columns
Columns: ['ResponseId', 'WebframeWantToWorkWith']


,ResponseId,WebframeWantToWorkWith
0,2,Express
1,2,Htmx
2,2,Node.js
3,2,React
4,2,Remix
5,3,ASP.NET CORE
6,10,Express
7,10,Next.js
8,10,Node.js
9,10,React




split_WebframeHaveWorkedWith.csv - First 10 Record:
------------------------------------------------------------
Dimension: 77803 rows, 2 columns
Columns: ['ResponseId', 'WebframeHaveWorkedWith']


,ResponseId,WebframeHaveWorkedWith
0,2,Express
1,2,Next.js
2,2,Node.js
3,2,React
4,3,ASP.NET CORE
5,10,Express
6,10,Next.js
7,10,Node.js
8,10,React
9,11,Node.js


Now, this csv files are ready to help creating these dashboards:

**Current Technology Usage - 2 x 2:**

Panel 1: Top 10 Languages Used (Stacked Bar)

Panel 2: Top 10 Databases Used (Stacked Column)

Panel 3: Top 10 Platforms Used (Word Cloud)

Panel 4: Top 10 Web Frameworks Used (Scatter Bubble)

**Future Technology Trends -  2 x 2 :**

Panel 1: Top 10 Languages Desired Next Year (Stacked Bar)

Panel 2: Top 10 Databases Desired Next Year (Stacked Column)

Panel 3: Top 10 Desired Platforms (Tree Map)

Panel 4: Top 10 Desired Web Frameworks (Scatter Bubble)

**Demographics - 2 x 2:**

Panel 1: Respondents by Age (Pie Chart)

Panel 2: Respondent Count by Country (Map Chart)

Panel 3: Respondent Distribution by Education Level (Line Bar Chart)

Panel 4: Respondent Count by Age, Classified by Education Level (Stacked bar Chart)